In [38]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats("svg")

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei'] # 指定默认字体为微软雅黑
plt.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

In [39]:
dataDir = "/home/twh/code/mtba/asap/data"


def get_data_path(dataFile):
    return os.path.join(dataDir, dataFile)


def get_data(dataFile):
    return pd.read_csv(get_data_path(dataFile))

In [40]:
train_set = get_data("train.csv")
test_set = get_data("test.csv")
print(train_set.columns)

Index(['id', 'review', 'star', 'Location#Transportation', 'Location#Downtown',
       'Location#Easy_to_find', 'Service#Queue', 'Service#Hospitality',
       'Service#Parking', 'Service#Timely', 'Price#Level',
       'Price#Cost_effective', 'Price#Discount', 'Ambience#Decoration',
       'Ambience#Noise', 'Ambience#Space', 'Ambience#Sanitary', 'Food#Portion',
       'Food#Taste', 'Food#Appearance', 'Food#Recommend'],
      dtype='object')


In [ ]:
train_lengths = [len(review) for review in train_set["review"]]
test_lengths = [len(review) for review in test_set["review"]]

plt.hist(train_lengths, bins=30, alpha=0.6, label="train", color="dodgerblue")
plt.hist(test_lengths, bins=30, alpha=0.6, label="test", color="greenyellow")
plt.xlabel("评论字数")
plt.ylabel("数量")
plt.legend()